In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="courtlin-udacity",subscription_id="bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8", resource_group="udacity-nano")
exp = Experiment(workspace=ws, name="courtlin-udacity")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: courtlin-udacity
Azure region: eastus2
Subscription id: bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8
Resource group: udacity-nano


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute import ComputeTarget, AmlCompute
cpu_cluster_name = "udacityproj1"
compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5, vm_priority='lowpriority',min_nodes=0, idle_seconds_before_scaledown=300)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice

ps = RandomParameterSampling( {
        "learning_rate": normal(10, 3),
        "keep_probability": uniform(0.05, 0.1),
        "batch_size": choice(16, 32, 64, 128)
    }
)
# Specify an Early Termination Policy
# policy = ### YOUR CODE HERE ###
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###


from azureml.core import ScriptRunConfig

# import os
# est = SKLearn(source_directory=os.path.join('./'),
#                       entry_script='train.py',
#                       compute_target=cpu_cluster,
#              )

compute_target = ws.compute_targets['udacityproj1']

# from azureml.core import Environment
# sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

# src = ScriptRunConfig(source_directory=os.path.join('./'),
#                       script='train.py',
#                       arguments=['--kernel', 'linear', '--penalty', 1.0],
#                       compute_target=compute_target,
#                       environment=sklearn_env)

# est = SKLearn(source_directory=os.path.join('./'),
#                       entry_script='train.py',
#                       compute_target=cpu_cluster,
#              )

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=".",
    compute_target=compute_target,
    entry_script="train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
                             estimator=est,
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=10)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

from azureml.core.experiment import Experiment
experiment = Experiment(ws, "courtlin-udacity")
hyperdrive_run = experiment.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
# from azureml.widgets import RunDetails
# RunDetails(hyperdrive_run).show()

In [6]:
import joblib
from azureml.core.model import Model

# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

# model = best_run.register_model(model_name='hyperdrive_model', model_path='./outputs/model.pkl', model_framework=Model.Framework.SCIKITLEARN, model_framework_version='0.19.1')

### YOUR CODE HERE ###

# best_run = hyperdrive_run.get_best_run_by_primary_metric()
# best_run_metrics = best_run.get_metrics()
# parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

from azureml.core.dataset import Dataset

url_paths = [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
]

ds = Dataset.Tabular.from_delimited_files(path=url_paths)
ds.to_pandas_dataframe()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###